In [1]:
# 🔧 Import libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# 📂 Prepare the dataset (update path if needed)
data_dir = r'C:\Users\admin\Desktop\yapa zeka\deepfake_split_dataset'

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
import torch
import torch.nn as nn

class MyCNN(nn.Module):
    """
    A deep residual CNN for deepfake detection, encapsulated in a single MyCNN class.
    """
    class ResidualBlock(nn.Module):
        def __init__(self, in_channels, out_channels, stride=1, downsample=None, drop_rate=0.3):
            super().__init__()
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                                   stride=stride, padding=1, bias=False)
            self.bn1   = nn.BatchNorm2d(out_channels)
            self.relu  = nn.ReLU(inplace=True)
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                                   stride=1, padding=1, bias=False)
            self.bn2   = nn.BatchNorm2d(out_channels)
            self.drop  = nn.Dropout(drop_rate)
            self.downsample = downsample

        def forward(self, x):
            identity = x

            out = self.conv1(x)
            out = self.bn1(out)
            out = self.relu(out)
            out = self.drop(out)
            out = self.conv2(out)
            out = self.bn2(out)

            if self.downsample is not None:
                identity = self.downsample(x)

            out += identity
            out = self.relu(out)
            return out

    def __init__(self, num_classes=2, drop_rate=0.3):
        super().__init__()
        # Stem: initial convolution and pooling
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2, padding=1)
        )
        # Residual stages
        self.layer1 = self._make_layer(64,  64,  blocks=2, stride=1,  drop_rate=drop_rate)
        self.layer2 = self._make_layer(64, 128, blocks=2, stride=2,  drop_rate=drop_rate)
        self.layer3 = self._make_layer(128,256, blocks=2, stride=2,  drop_rate=drop_rate)
        self.layer4 = self._make_layer(256,512, blocks=2, stride=2,  drop_rate=drop_rate)

        # Head: pooling and classification
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(drop_rate),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(drop_rate),
            nn.Linear(256, num_classes)
        )

    def _make_layer(self, in_channels, out_channels, blocks, stride, drop_rate):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )
        layers = [MyCNN.ResidualBlock(in_channels, out_channels, stride, downsample, drop_rate)]
        for _ in range(1, blocks):
            layers.append(MyCNN.ResidualBlock(out_channels, out_channels, drop_rate=drop_rate))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [7]:
from tqdm import tqdm  # استيراد مكتبة tqdm

# 🚀 Train the model
model = MyCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 15  # على سبيل المثال 5 epochs
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train, total_train = 0, 0
    
    # استخدام tqdm لعرض شريط تقدم أثناء التدريب
    with tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}', unit='batch', ncols=100) as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # حساب الدقة للتدريب
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)
            
            # تحديث شريط التقدم مع معلومات الدقة والخسارة
            train_accuracy = 100 * correct_train / total_train
            pbar.set_postfix(train_loss=running_loss / (total_train // len(labels)), train_acc=f'{train_accuracy:.4f}%')
    
    train_losses.append(running_loss / len(train_loader))

    model.eval()
    val_loss, correct, total = 0.0, 0, 0

    # استخدام tqdm لعرض شريط تقدم أثناء التحقق
    with tqdm(val_loader, desc=f'Validation Epoch {epoch+1}/{num_epochs}', unit='batch', ncols=100) as pbar:
        with torch.no_grad():
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                # حساب الدقة للتحقق
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                # تحديث شريط التقدم مع معلومات الدقة والخسارة
                val_accuracy = 100 * correct / total
                pbar.set_postfix(val_loss=val_loss / (total // len(labels)), val_acc=f'{val_accuracy:.4f}%')
    
    val_losses.append(val_loss / len(val_loader))
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_losses[-1]:.4f}, Val Acc: {acc:.2f}%")


Training Epoch 1/15: 100%|█| 4218/4218 [05:18<00:00, 13.26batch/s, train_acc=86.5563%, train_loss=0.
Validation Epoch 1/15: 100%|█| 528/528 [00:37<00:00, 14.03batch/s, val_acc=96.4552%, val_loss=0.0162


Epoch 1/15 - Train Loss: 0.2889, Val Acc: 96.46%


Training Epoch 2/15: 100%|█| 4218/4218 [05:06<00:00, 13.74batch/s, train_acc=96.2128%, train_loss=0.
Validation Epoch 2/15: 100%|█| 528/528 [00:36<00:00, 14.55batch/s, val_acc=97.7593%, val_loss=0.0090


Epoch 2/15 - Train Loss: 0.0932, Val Acc: 97.76%


Training Epoch 3/15: 100%|█| 4218/4218 [05:11<00:00, 13.54batch/s, train_acc=97.3732%, train_loss=0.
Validation Epoch 3/15: 100%|█| 528/528 [00:36<00:00, 14.47batch/s, val_acc=97.8601%, val_loss=0.0093


...

Epoch 13/15 - Train Loss: 0.0260, Val Acc: 98.45%


Training Epoch 14/15: 100%|█| 4218/4218 [05:46<00:00, 12.18batch/s, train_acc=98.5817%, train_loss=0
Validation Epoch 14/15: 100%|█| 528/528 [00:43<00:00, 12.16batch/s, val_acc=98.5003%, val_loss=0.004


Epoch 14/15 - Train Loss: 0.0252, Val Acc: 98.50%


Training Epoch 15/15: 100%|█| 4218/4218 [05:27<00:00, 12.87batch/s, train_acc=98.5981%, train_loss=0
Validation Epoch 15/15: 100%|█| 528/528 [00:38<00:00, 13.67batch/s, val_acc=98.5122%, val_loss=0.005

Epoch 15/15 - Train Loss: 0.0238, Val Acc: 98.51%


In [8]:
# 🧪 Test accuracy
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.11%


In [11]:
# 💾 Save the model
torch.save(model.state_dict(), 'deepfake_model.pth')
print("Model saved as deepfake_model.pth")

Model saved as deepfake_model.pth
